In [1]:
import json
import numpy as np
from sklearn.cluster import KMeans

In [4]:
infernece_result_path = "/home/srini/workspace/BLOOM/textTOSQL_Paper/EHRSQL/EHRSQL/decoder_outputs/mistral7B/test_vllm_main_create_prompt_2.json"


In [5]:
num_workers = -1
with open(infernece_result_path, 'r') as f:
    data = json.load(f)
print(f'[result] {len(data)} lines loaded')

data_id = []
query_real = []
query_pred = []
entropy = []
impossible = []
for idx_, line in data.items():
    data_id.append(idx_)
    query_real.append(line['real'])
    query_pred.append(line['pred'])
    entropy.append(max(line['sequence_entropy']))
    impossible.append(line['is_impossible'])

[result] 1122 lines loaded


# k-means

In [6]:
def get_threshold_kmeans(entropy):
    
    kmeans = KMeans(n_clusters=2, random_state=0).fit(np.expand_dims(entropy, axis=1))
    
    zero_low = min(np.array(entropy)[kmeans.labels_==0])
    zero_high = max(np.array(entropy)[kmeans.labels_==0])
    one_low = min(np.array(entropy)[kmeans.labels_==1])
    one_high = max(np.array(entropy)[kmeans.labels_==1])
    
    if one_high > zero_high:
        return (zero_high + one_low)/2
    else:
        return (zero_low + one_high)/2

In [7]:
get_threshold_kmeans(entropy)

/home/srini/miniconda3/envs/text2SQL/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1.3507996797561646

In [16]:
# mimic3 k-means: 0.744074136018753
# mimic3_schema k-means: 0.8807361125946045

# percentile

In [25]:
np.percentile(entropy, q=[67.0])

array([0.1141332])

In [18]:
# mimic3 67 percentile: 0.14923561
# mimic3_schema 67 percentile: 0.09932577

# choosing a threshold

In [19]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold -1
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

usage: abstain_with_entropy.py [-h] --inference_result_path
                               INFERENCE_RESULT_PATH [--input_file INPUT_FILE]
                               [--output_file OUTPUT_FILE]
                               [--threshold THRESHOLD]
abstain_with_entropy.py: error: the following arguments are required: --inference_result_path


^C
Traceback (most recent call last):
  File "/home/srini/workspace/BLOOM/textTOSQL_Paper/EHRSQL/EHRSQL/evaluate.py", line 176, in <module>
    main(args)
  File "/home/srini/workspace/BLOOM/textTOSQL_Paper/EHRSQL/EHRSQL/evaluate.py", line 123, in main
    execute_query_distributed(query_real, query_pred, args.db_path, num_workers)
  File "/home/srini/workspace/BLOOM/textTOSQL_Paper/EHRSQL/EHRSQL/evaluate.py", line 98, in execute_query_distributed
    pool.join()
  File "/home/srini/miniconda3/envs/text2SQL/lib/python3.10/multiprocessing/pool.py", line 662, in join
    self._worker_handler.join()
  File "/home/srini/miniconda3/envs/text2SQL/lib/python3.10/threading.py", line 1089, in join
    self._wait_for_tstate_lock()
  File "/home/srini/miniconda3/envs/text2SQL/lib/python3.10/threading.py", line 1105, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.744074136018753
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 83.84,
  "recall_ans": 97.63,
  "f1_ans": 90.21,
  "precision_exec": 82.03,
  "recall_exec": 95.53,
  "f1_exec": 88.27
}


In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.14923561
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 94.81,
  "recall_ans": 93.82,
  "f1_ans": 94.31,
  "precision_exec": 93.88,
  "recall_exec": 92.89,
  "f1_exec": 93.39
}


In [ ]:
# high-precision setting

In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.1
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 95.82,
  "recall_ans": 93.42,
  "f1_ans": 94.6,
  "precision_exec": 95.14,
  "recall_exec": 92.76,
  "f1_exec": 93.94
}


In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.000050
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 98.46,
  "recall_ans": 8.42,
  "f1_ans": 15.52,
  "precision_exec": 98.46,
  "recall_exec": 8.42,
  "f1_exec": 15.52
}


In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.000040
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 97.78,
  "recall_ans": 5.79,
  "f1_ans": 10.93,
  "precision_exec": 97.78,
  "recall_exec": 5.79,
  "f1_exec": 10.93
}


In [ ]:
!python T5/abstain_with_entropy.py --infernece_result_path outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid --input_file prediction_raw.json --output_file prediction.json --threshold 0.000030
!python evaluate.py --db_path ./dataset/ehrsql/mimic_iii/mimic_iii.db --data_file dataset/ehrsql/mimic_iii/valid.json --pred_file ./outputs/eval_ehrsql_mimic3_t5_base__mimic3_valid/prediction.json

{
  "precision_ans": 100.0,
  "recall_ans": 3.68,
  "f1_ans": 7.11,
  "precision_exec": 100.0,
  "recall_exec": 3.68,
  "f1_exec": 7.11
}


In [ ]:
# precision_exec: 100.0 (valid) >= 99.0
# set the threshold to 0.000030 and now ready to evaluate on the test set.